# 1

In [11]:
import pandas as pd

In [10]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
data.head()

,Postcode,District,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
data = data.rename(columns={'Postcode': 'PostalCode','District':'Borough'})

In [13]:
data = data.loc[(data['Borough'] != 'Not assigned')]

In [14]:
data.loc[data['Neighbourhood'] == 'Not assigned','Neighbourhood'] = data.loc[data['Neighbourhood'] == 'Not assigned','Borough']

In [16]:
neighborhoods = data.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
neighborhoods.shape

(103, 3)

In [17]:
neighborhoods.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# 2

In [29]:
geodata_toronto = pd.read_csv('http://cocl.us/Geospatial_data')
geodata_toronto.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
neighborhoods = pd.merge(neighborhoods, geodata_toronto,left_on="PostalCode", right_on="Postal Code")

In [31]:
neighborhoods.shape

(103, 6)

In [32]:
neighborhoods.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [33]:
neighborhoods = neighborhoods.drop('Postal Code',axis = 1)

In [34]:
neighborhoods.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# 3

In [39]:
# uncomment this line if you haven't completed the Foursquare API lab
#! pip install folium
import folium # map rendering library

     |████████████████████████████████| 92kB 11.4MB/s eta 0:00:01


In [ ]:
#address = 'New York City, NY'

#geolocator = Nominatim(user_agent="ny_explorer")
#location = geolocator.geocode(address)
#latitude = location.latitude
#longitude = location.longitude
#print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

In [46]:
latitude = 43.653908
longitude = -79.384293
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto